In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, time
import argparse
import tarfile
import re
import scipy.stats as stats

##impor  BMD files from directory
##TODO: combine bMD processing to single file, combine LPR processing to single file
import bmd_analysis_morpho as bmd
import bmd_analysis_LPR_7_PAH_t0_t239 as bmd_LPR
import format_LPR_input as format_LPR
import format_morpho_input as format_morpho

# Journey that a single morpho file takes

In [ ]:
mfiles = './test_files/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv'

morpho_input_csv_file_name_wide = mfiles.split("/")[-1] + "_wide_DNC_0_"+ "fd" + ".csv"

In [ ]:
df_morph = pd.read_csv(mfiles, header = 0)

# Keep onloy relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
df_morph_select = df_morph.loc[:,columns_to_keep]

# Convert numerics to strings 
df_morph_select['plate.id'] = df_morph_select['plate.id'].astype(str)

In [ ]:
df_morph = pd.read_csv(mfiles, header = 0)
relevant_columns = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
all(col in df_morph.columns for col in relevant_columns) == False
print("The input file", mfiles, "must have the columns:", ', '.join(relevant_columns)) 

In [ ]:
df_morph_select.dtypes

In [ ]:
df_reformatted = pd.DataFrame()

for chemical_index in pd.unique(df_morph['chemical.id']):

    print("chemical_index:" + str(chemical_index))
    #total_number_of_unique_chemicals += 1
    morph_data_chemical = df_morph_select.loc[df_morph['chemical.id'] == chemical_index,:]

    # Append chemical_plate_well as a unique identifier
    morph_data_chemical.insert(0, 'chemical_plate_well', morph_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))

    for cpw in np.unique(morph_data_chemical.chemical_plate_well):
        #total_number_of_chemical_plate_well += 1
        temp_df = morph_data_chemical.loc[morph_data_chemical.chemical_plate_well == cpw,:]

        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in temp_df_grouped:

            ## JUSTIFICATION: 7 PAH dataset doesn't have "BRAI" endpoint.
            ## On the other hand, extracts/phase I,II have "BRAI" endpoint.
            if 'BRAI' not in temp_df.endpoint.values: # as 7 PAH
                try:
                    #            if(len(group.endpoint) == 14):
                    temp = pd.DataFrame( {
                        'chemical.id': pd.unique(temp_df['chemical.id']),
                        'plate.id': pd.unique(temp_df['plate.id']),
                        'well': pd.unique(temp_df['well']),
                        'chemical_plate_well': pd.unique(temp_df['chemical_plate_well']),
                        'conc': pd.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRN_': temp_df.value[temp_df.endpoint == 'BRN_'].values,
                        'CRAN': temp_df.value[temp_df.endpoint == 'CRAN'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EDEM': temp_df.value[temp_df.endpoint == 'EDEM'].values,
                        'LTRK': temp_df.value[temp_df.endpoint == 'LTRK'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'MUSC': temp_df.value[temp_df.endpoint == 'MUSC'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'SKIN': temp_df.value[temp_df.endpoint == 'SKIN'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'TCHR': temp_df.value[temp_df.endpoint == 'TCHR'].values,
                    }  )
                    df_reformatted = pd.concat([df_reformatted, temp])
                except:
                    print ("len(group.endpoint) != 14")
                    print ("chemical_plate_well:" + str(cpw))
            else: #as extracts
                temp = pd.DataFrame(
                    {
                    'chemical.id': pd.unique(temp_df['chemical.id']),
                    'plate.id': pd.unique(temp_df['plate.id']),
                    'well': pd.unique(temp_df['well']),
                    'chemical_plate_well': pd.unique(temp_df['chemical_plate_well']),
                    'conc': pd.unique(temp_df['conc']),
                    'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                    'BRAI': temp_df.value[temp_df.endpoint == 'BRAI'].values,
                    'CFIN': temp_df.value[temp_df.endpoint == 'CFIN'].values,
                    'CIRC': temp_df.value[temp_df.endpoint == 'CIRC'].values,
                    'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                    'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                    'EYE_': temp_df.value[temp_df.endpoint == 'EYE_'].values,
                    'JAW_': temp_df.value[temp_df.endpoint == 'JAW_'].values,
                    'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                    'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                    'NC24': temp_df.value[temp_df.endpoint == 'NC24'].values,
                    'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                    'OTIC': temp_df.value[temp_df.endpoint == 'OTIC'].values,
                    'PE__': temp_df.value[temp_df.endpoint == 'PE__'].values,
                    'PFIN': temp_df.value[temp_df.endpoint == 'PFIN'].values,
                    'PIG_': temp_df.value[temp_df.endpoint == 'PIG_'].values,
                    'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                    'SNOU': temp_df.value[temp_df.endpoint == 'SNOU'].values,
                    'SOMI': temp_df.value[temp_df.endpoint == 'SOMI'].values,
                    'SWIM': temp_df.value[temp_df.endpoint == 'SWIM'].values,
                    'TRUN': temp_df.value[temp_df.endpoint == 'TRUN'].values,
                    'TR__': temp_df.value[temp_df.endpoint == 'TR__'].values,
                    'YSE_': temp_df.value[temp_df.endpoint == 'YSE_'].values,
                    })
                df_reformatted = pd.concat([df_reformatted, temp])
#print ("total_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))
#print ("total_number_of_chemical_plate_well:" + str(total_number_of_chemical_plate_well))
#end_time = time.time()
#time_took = str(round((end_time-start_time), 1)) + " seconds"
#print ("Done, it took:"+str(time_took))

pd.set_option('display.max_columns', None)
print (df_reformatted.head())
#print ("df_reformatted.shape:" + str(df_reformatted.shape)) #(288, 19)

df_reformatted_DNC_0 = pd.DataFrame()

df_reformatted_DNC_0 = df_reformatted.loc[df_reformatted['DNC_'] == 0.0]
print ("df_reformatted_DNC_0.shape:" + str(df_reformatted_DNC_0.shape)) #(287, 19)

df_reformatted_DNC_0.to_csv("./Reformatted.csv", index=False)

In [ ]:
# Pivot long format table wider, and sum up the ones 
pivot_wider = df_morph_select.pivot_table(index = ['chemical.id', 'plate.id', 'well', 'conc'], columns = ['endpoint'], values = ['value'], aggfunc = 'first')

# Convert to a clean pandas dataframe 
pivot_wider = pd.DataFrame(pivot_wider.to_records())

# Clean up the column names 
new_colnames = []

# Fix column names with the extra 'value' annotation. Otherwise, ignore.
for name in pivot_wider.columns:
    if "value" in name:
        new_colnames.append(re.sub("\\(value,|\\)", "", re.sub("'", "", name)).strip())
    else:
        new_colnames.append(name)

# Rename columns
pivot_wider = pivot_wider.set_axis(new_colnames, axis = 1, inplace = False)

In [ ]:
relevant_columns = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
for x in ['AXIS', 'BRN_', 'CRAN', 'DNC_', 'DP24', 'EDEM', 'LTRK', 'MO24', 'MORT', 'MUSC', 'NC__', 'SKIN','SM24', 'TCHR']:
    relevant_columns.append(x)
relevant_columns

In [ ]:
df_morph.pivot_table(index = ['chemical.id', 'plate.id', 'well', 'conc'], columns = ['endpoint'], 
                                       values = ['value'], aggfunc = 'first')

# Morpho Step 1: Format File

In [140]:
# This is the format_morpho_input function

##############################################
## READ FILE AND SUBSET TO RELEVANT COLUMNS ##
##############################################

# Read morphology file 
df_morph = pd.read_csv('./test_files/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv', header = 0)

# List relevant column names
relevant_columns = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']

# The input file must absolutely have these columns, no exceptions 
if all(col in df_morph.columns for col in relevant_columns) == False:
    sys.exit(print("The input file", mfiles, "must have the columns:", ', '.join(relevant_columns)))

# Keep only relevant columns
df_morph = df_morph.loc[:,relevant_columns]

##################################
## SUBSET TO RELEVANT ENDPOINTS ##
##################################

# List the relevant endpoints, which is different for BRAIN samples  
if "BRAI" in list(df_morph["endpoint"].unique()):
    relevant_endpoints = ['AXIS', 'BRAI', 'CFIN', 'CIRC', 'DNC_', 'DP24', 'EYE_', 'JAW_', 'MO24', 
                          'MORT', 'NC24', 'NC__', 'OTIC', 'PE__', 'PFIN', 'PIG_', 'SM24', 'SNOU', 
                          'SOMI', 'SWIM', 'TRUN', 'TR__', 'YSE_']
else:
    relevant_endpoints = ['AXIS', 'BRN_', 'CRAN', 'DNC_', 'DP24', 'EDEM', 'LTRK', 'MO24', 'MORT', 
                          'MUSC', 'NC__', 'SKIN','SM24', 'TCHR']
    
# Subset down to the relevant endpoints 
df_morph = df_morph[df_morph["endpoint"].isin(relevant_endpoints)]

###########################
## ADD MISSING ENDPOINTS ##
###########################

def new_endpoint(endpoints, new_name):
    """
    Generate a new endpoint which is a binary "or" of other endpoints,
    meaning that if there is a 1 in any of the other endpoints, the 
    resulting endpoint is a 1. Otherwise, it is 0 unless the other 
    endpoints are all NA. Then the final value is NA.
    
    Attributes
    ----
    endpoints: A list of column names, as strings, to binary "or". 
    new_name: The name of the new endpoint. 
    
    """
    sub_df = df_morph[df_morph["endpoint"].isin(endpoints)]
    sub_df["endpoint"] = new_name
    sub_df = sub_df.groupby(by = ["chemical.id", "conc", "plate.id", "well", "endpoint"], as_index = False).sum()
    sub_df['value'].values[sub_df['value'] > 1] = 1 
    return(sub_df)
    
# New endpoints to add is a smaller list if the sample is not from BRAIN
if "BRAI" in list(chemical_groups["endpoint"].unique()):
    
    df_morph = pd.concat(
        [new_endpoint(['MO24','DP24','SM24','NC24'], 'ANY24'),
         new_endpoint(['MORT', 'YSE_', 'AXIS', 'EYE_', 'SNOU', 'JAW_', 'OTIC', 'PE__', 'BRAI', 
                      'SOMI', 'PFIN', 'CFIN', 'PIG_', 'CIRC', 'TRUN', 'SWIM', 'NC__', 'TR__', 
                      'ANY24'], 'ANY120'),
         new_endpoint(['MO24','MORT'], 'TOT_MORT'),
         new_endpoint(['DP24','SM24','NC24', 'YSE_', 'AXIS', 'EYE_', 'SNOU', 'JAW_', 'OTIC', 'PE__', 
                      'BRAI', 'SOMI', 'PFIN', 'CFIN', 'PIG_', 'CIRC','TRUN', 'SWIM', 'NC__', 'TR__'], 'ALL_BUT_MORT'),
         new_endpoint(['BRAI','OTIC','PFIN'], 'BRN_'),
         new_endpoint(['EYE_', 'SNOU', 'JAW_'], 'CRAN'),
         new_endpoint(['YSE_','PE__'], 'EDEM'),
         new_endpoint(['TRUN','CFIN'], 'LTRK'),
         new_endpoint(['CIRC','SWIM','SOMI'], 'MUSC'),
         new_endpoint(['PIG_'], 'SKIN'),
         new_endpoint(['TR__'], 'TCHR'),
         df_morph]
    )
    
else:
    
    df_morph = pd.concat(

        # 1. Add any effect at 24hrs (combination of MO24, DP24 and SM24) 
        [new_endpoint(['MO24','DP24','SM24'], 'ANY24'),

        # 2. Any effect within 5 days (combination of all measurements at both time points)
        new_endpoint(['AXIS', 'BRN_', 'CRAN', 'EDEM', 'LTRK', 'MORT', 'MUSC', 'NC__', 'SKIN', 'TCHR', 'ANY24'], 'ANY120'),

        # 3. Total mortality (MO24 + MORT) 
        new_endpoint(['MO24','MORT'], 'TOT_MORT'),

        # 4. Any effect except mortality (#2 minus MO24 and MORT)
        new_endpoint(['AXIS', 'BRN_', 'CRAN', 'DP24', 'EDEM', 'LTRK', 'MUSC', 'NC__', 'SKIN', 'SM24', 'TCHR'], 'ALL_BUT_MORT'),
        
        # Add original dataframe
        df_morph]
    )
    
###########################################
## CALCULATE VARIABLES FOR DOSE RESPONSE ##
###########################################

# Create groups of each chemical id, concentration, and plate id
plate_groups = df_morph.drop(["well"], 1).groupby(by = ["chemical.id", "conc", "plate.id", "endpoint"], as_index = False)

# Get the number of samples per group
num_tot_samples = plate_groups.size().rename(columns = {"size": "num.tot"})

# Get the number of non-na samples per groups
num_nonna = plate_groups.count().rename(columns = {"value": "num.nonna"})

# Get the number affected
num_affected = plate_groups.sum().rename(columns = {"value": "num.affected"})

# Merge to create missingness dataframe
plate_groups = pd.merge(pd.merge(num_tot_samples, num_nonna), num_affected)

# Create IDs of chemical.id, plate.id, and endpoint in plate_groups 
ids = []
for row in range(len(plate_groups)):
    ids.append(str(plate_groups["chemical.id"][row]) + " " + str(plate_groups["plate.id"][row]) + " " + str(plate_groups["endpoint"][row]))
plate_groups["ids"] = ids

#####################################################################
## REMOVE VARIABLES WITH HIGH MISSINGNESS IN BASELINE MEASUREMENTS ##
#####################################################################

# Identify 0 (baseline) concentrations with high missingness (greater than 50% missing or less than 50% non-missing)
missingness = plate_groups.loc[plate_groups["conc"] == 0]
missingness["keep"] = missingness["num.nonna"] / missingness["num.tot"] > 0.5 # TODO: Add a report of what was removed --> txt file "nothing removed"

# Identify plates to keep 
tokeep = missingness.loc[missingness["keep"]]["ids"].tolist()
plate_groups = plate_groups[plate_groups["ids"].isin(tokeep)]

# Stop if everything gets removed
if len(plate_groups) == 0:
    sys.exit("Everything was removed with the 50% missingness filter")

#######################################
## REGROUP WITHOUT PLATE IDS AND SUM ##
#######################################

# First, remove plate.id and ids column
chemical_groups = plate_groups.drop(columns = ["plate.id", "ids"])

# Group by chemical.id, concentration, and endpoint. Then, sum the results. 
chemical_groups = chemical_groups.groupby(by = ["chemical.id", "conc", "endpoint"]).sum().reset_index()

############################
## RETURN FORMATTED TABLE ##
############################
chemical_groups


,chemical.id,conc,endpoint,num.tot,num.nonna,num.affected
0,3756,0.0,ALL_BUT_MORT,36,36,2.0
1,3756,0.0,ANY120,36,36,2.0
2,3756,0.0,ANY24,36,36,4.0
3,3756,0.0,AXIS,36,32,2.0
4,3756,0.0,BRN_,36,32,0.0
...,...,...,...,...,...,...
139,3756,100.0,NC__,36,30,0.0
140,3756,100.0,SKIN,36,30,0.0
141,3756,100.0,SM24,36,32,0.0
142,3756,100.0,TCHR,36,30,0.0


# Morpho Step 2: Generate Dose Response and Write Flags

In [162]:
# This is the generate dose response function

'''This function performs feasibility analysis
for dose response data. The value returned is a
flag indicating data quality as defined below:

0: Not enough dose groups for BMD analysis. BMD analysis not performed.
1: No trend detected in dose-response data. BMD analysis not performed.
2: Good dose-response data. BMD analysis is performed.
3: Dose-response data quality poor. BMD analysis might be unreliable.
4: Data resolution poor. BMD analysis might be unreliable.
5: No trend detected in dose-response data. BMD analysis not performed

'''

########################
## CALCULATE RESPONSE ##
########################

# Response is the number affected over the number of embryos (non-na)
dose_response = chemical_groups
dose_response["frac.affected"] = dose_response["num.affected"] / dose_response["num.nonna"]

####################################
## GENERATE QUALITY CONTROL FLAGS ##
####################################

# Count the number of unique concentrations per chemical id and endpoint pairing 
BMD_Flags = dose_response.groupby(["chemical.id", "endpoint"])["conc"].nunique().reset_index().rename(columns = {"conc": "num.conc"})

# Add a flag category
BMD_Flags["flag"] = None

# If there are less than 3 points, the BMD flag is 0 - not enough dose groups
BMD_Flags["flag"].values[BMD_Flags["num.conc"] < 3] = 0

# Change dose response 
dose_response["conc"] = dose_response["conc"].astype('float') + 1e-15

# Calculate the spearman correlation
spear = dose_response[["chemical.id", "endpoint", "conc", "frac.affected"]].groupby(["chemical.id", "endpoint"]).corr(method = "spearman").unstack().iloc[:,1].reset_index()
spear = spear.set_axis(["chemical.id", "endpoint", "spearman"], axis = 1)

# Merge spearman to the BMD_Flags dataframe
BMD_Flags = BMD_Flags.merge(spear)

# If the correlation is above 0.25 or below 0.8, run a t-test
BMD_Flags["run.ttest"] = (BMD_Flags["spearman"] > 0.25) & (BMD_Flags["spearman"] < 0.8)

BMD_Flags


,chemical.id,endpoint,num.conc,flag,spearman,run.ttest
0,3756,ALL_BUT_MORT,8,None,0.739516,True
1,3756,ANY120,8,None,0.566306,True
2,3756,ANY24,8,None,0.727393,True
3,3756,AXIS,8,None,0.357143,True
4,3756,BRN_,8,None,NaN,False
5,3756,CRAN,8,None,0.357143,True
6,3756,DNC_,8,None,0.247436,False
7,3756,DP24,8,None,0.805118,False
8,3756,EDEM,8,None,0.357143,True
9,3756,LTRK,8,None,0.000000,False


In [ ]:
    if (global_report):
        print ("dose_response:\n"+str(dose_response))
    if(dose_response.shape[0] < 3):
        BMD_feasibilitye_flag = 0
    else:
        frac_response = dose_response['num_affected']/dose_response['num_embryos']
        #frac_response = dose_response['num_affected']/dose_response['frac_affect']

        if (global_report):
            print ("dose_response['num_affected']:\n" + str(dose_response['num_affected']))
            print ("dose_response['num_embryos']:\n" + str(dose_response['num_embryos']))
            print ("frac_response:\n" + str(frac_response))
            print ("frac_response[0]:\n" + str(frac_response[0]))
            print ("data_corr:" + str(data_corr))
            print ("data_corr[0]:" + str(data_corr[0]))